In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
from IPython.core.display import display, HTML, Javascript

html_contents ="""
<!DOCTYPE html>
<html lang="en">
    <head>
        <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
        <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Raleway">
        <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Oswald">
        <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Open Sans">
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
        <style>
        .title-section{
            font-family: "Roboto", Verdana, sans-serif;
            font-weight: bold;
            color: "#8B008B";
            letter-spacing: 8px;
        }
        hr { border: 1px solid #E58F65 !important;
             color: #E58F65 !important;
             background: #E58F65 !important;
           }
        body {
            font-family: "Open Sans", sans-serif;
            }        
        </style>
    </head>    
</html>
"""

HTML(html_contents)


#  <span class="title-section w3-xxlarge" style="color:#FF0080">Tokyo Stock Exchange (JPX) </span>
![JPX](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Japan_Exchange_Group_logo.svg/75px-Japan_Exchange_Group_logo.svg.png)
![JPX](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Tokyo_Stock_Exchange_logo.svg/150px-Tokyo_Stock_Exchange_logo.svg.png)

### <span class="title-section w3-large" style="color:magenta"> The Tokyo Stock Exchange (東京証券取引所, Tōkyō Shōken Torihikijo), abbreviated as Tosho (東証) or TSE/TYO, is a stock exchange located in Tokyo, Japan. It is the third largest stock exchange in the world by aggregate market capitalization of its listed companies, and the largest in Asia.</span>

### <span class="title-section w3-large" style="color:green"> JPX/TSE had 2,292 listed companies with a combined market capitalization of US$5.67 trillion as of February 2019. The exchange is owned by the Japan Exchange Group (JPX), a holding company that it also lists (TYO: 8697). JPX was formed from its merger with the Osaka Exchange; the merger process began in July 2012, when said merger was approved by the Japan Fair Trade Commission </span>

### <span class="title-section w3-large" style="color:blue"> JPX itself was launched on January 1, 2013</span>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
from pandas_datareader.data import DataReader
   
# For time stamps
from datetime import datetime
from math import sqrt
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import spearmanr
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)

import shap
import xgboost
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm.auto import tqdm

#ignore the warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
FILE_DIR="../input/jpx-tokyo-stock-exchange-prediction/data_specifications/"
stock_file_inputs = {
    "Options_File": f"{FILE_DIR}/options_spec.csv",
    "Stock_Fin_File": f"{FILE_DIR}/stock_fin_spec.csv",
    "Stock_List_File": f"{FILE_DIR}/stock_list_spec.csv",
    "Stock_Price_File": f"{FILE_DIR}/stock_price_spec.csv",
    "Trades_File": f"{FILE_DIR}/trades_spec.csv",
}

Stock_File_Specs = {}
for NAME, STOCK in stock_file_inputs.items():
    print(NAME)
    Stock_File_Specs[NAME] = pd.read_csv(STOCK)
    print(Stock_File_Specs[NAME].head())
    print(Stock_File_Specs[NAME].shape)
    print(Stock_File_Specs[NAME].columns)

In [ ]:
TRAIN_DIR="../input/jpx-tokyo-stock-exchange-prediction/train_files/"
train_stock_file_inputs = {
    "Financials_File": f"{TRAIN_DIR}/financials.csv",
    "Options_File": f"{TRAIN_DIR}/options.csv",
    "Secondary_Stock_Price_File": f"{TRAIN_DIR}/secondary_stock_prices.csv",
    "Stock_Price_File": f"{TRAIN_DIR}/stock_prices.csv",
    "Trades_File": f"{TRAIN_DIR}/trades.csv",
}

Train_Stock_File_Specs = {}
for NAME, STOCK in train_stock_file_inputs.items():
    print(NAME)
    Train_Stock_File_Specs[NAME] = pd.read_csv(STOCK)
    print(Train_Stock_File_Specs[NAME].head())
    print(Train_Stock_File_Specs[NAME].shape)
    print(Train_Stock_File_Specs[NAME].columns)


In [ ]:

stock_list = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')
stock_list.columns

In [ ]:
stock_price = Train_Stock_File_Specs["Stock_Price_File"]
stocks=stock_price["SecuritiesCode"].unique()
print(stocks)
print(stocks.shape)
stock_price = Train_Stock_File_Specs["Stock_Price_File"]
print(stock_price.columns)

stock_name = stock_list[['SecuritiesCode', 'Name']]
merged_stock_price = pd.merge(stock_price,stock_name[['SecuritiesCode','Name']],on='SecuritiesCode', how='left')
merged_stock_price.head()


In [ ]:
merged_stock_price.head()

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
merged_stock_price_distr=merged_stock_price.groupby('SecuritiesCode').size().reset_index(name='total')
stock_list_data=pd.merge(stock_list,merged_stock_price_distr, how='left',on=['SecuritiesCode'])
stock_list_data=stock_list_data.groupby(['33SectorName']).total.sum().reset_index(name='total')
stock_list_data = stock_list_data.sort_values(by = 'total', ascending=False)
#print(stock_list_data.head())
sortx = sorted(stock_list_data["total"]) #[x for _,x in sorted(zip(stock_list_data["33SectorName"],stock_list_data['total']))]
sorty = sorted(stock_list_data["33SectorName"])
fig = px.bar(y = sorty, #stock_list_data["33SectorName"],
             x = sortx, #stock_list_data['total'] , 
             orientation = 'h',
             color = sorty, #stock_list_data["33SectorName"] ,
             color_continuous_scale="Viridis",
             width=1600, height=3000) 
fig.update_xaxes(title="Total Count")
fig.update_yaxes(title = "Type of Industry")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution ',
        'xanchor': 'center',
        'yanchor': 'top'} ,
        template="plotly_white")
fig.show()



In [ ]:
import plotly.express as px
import plotly.graph_objs as go
merged_stock_price_distr=merged_stock_price.groupby('SecuritiesCode').size().reset_index(name='total')
stock_list_data=pd.merge(stock_list,merged_stock_price_distr, how='left',on=['SecuritiesCode'])
stock_list_data=stock_list_data.groupby(['17SectorName']).total.sum().reset_index(name='total')
stock_list_data = stock_list_data.sort_values(by = 'total', ascending=False)
#print(stock_list_data.head())
sortx = sorted(stock_list_data["total"]) #[x for _,x in sorted(zip(stock_list_data["33SectorName"],stock_list_data['total']))]
sorty = sorted(stock_list_data["17SectorName"])
fig = px.bar(y = sorty, #stock_list_data["33SectorName"],
             x = sortx, #stock_list_data['total'] , 
             orientation = 'h',
             color = sorty, #stock_list_data["33SectorName"] ,
             color_continuous_scale="Viridis",
             width=1600, height=3000) 
fig.update_xaxes(title="Total Count")
fig.update_yaxes(title = "Type of Sector")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution ',
        'xanchor': 'center',
        'yanchor': 'top'} ,
        template="plotly_white")
fig.show()



In [ ]:
stock_list.head()
stock_list_simple = stock_list[['SecuritiesCode','Name','Section/Products','33SectorName','17SectorName','Close','IssuedShares','MarketCapitalization']]

In [ ]:
stock_list_simple.plot()

In [ ]:
#merged_stock_price_distr=merged_stock_price.groupby('SecuritiesCode').size().reset_index(name='total')
#stock_list_data=pd.merge(stock_list,merged_stock_price_distr, how='left',on=['SecuritiesCode'])
#stock_list_data=stock_list_data.groupby(['17SectorName']).total.sum().reset_index(name='total')
stock_list_data = stock_list_simple.sort_values(by = 'MarketCapitalization', ascending=False).head(100)
#print(stock_list_data.head())
sortx = sorted(stock_list_data["MarketCapitalization"]) #[x for _,x in sorted(zip(stock_list_data["33SectorName"],stock_list_data['total']))]
sorty = sorted(stock_list_data["Name"])
fig = px.bar(y = sorty, #stock_list_data["33SectorName"],
             x = sortx, #stock_list_data['total'] , 
             orientation = 'h',
             color =stock_list_data["33SectorName"], #stock_list_data["33SectorName"] ,
             color_continuous_scale="Viridis",
             width=1600, height=3000) 
fig.update_xaxes(title="Market Capitalization")
fig.update_yaxes(title = "Type of Sector")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution ',
        'xanchor': 'center',
        'yanchor': 'top'} ,
        template="plotly_white")
fig.show()


In [ ]:
#merged_stock_price_distr=merged_stock_price.groupby('SecuritiesCode').size().reset_index(name='total')
#stock_list_data=pd.merge(stock_list,merged_stock_price_distr, how='left',on=['SecuritiesCode'])
#stock_list_data=stock_list_data.groupby(['17SectorName']).total.sum().reset_index(name='total')
stock_list_data = stock_list_simple.sort_values(by = 'IssuedShares', ascending=False).head(100)
#print(stock_list_data.head())
sortx = sorted(stock_list_data["IssuedShares"]) #[x for _,x in sorted(zip(stock_list_data["33SectorName"],stock_list_data['total']))]
sorty = sorted(stock_list_data["Name"])
fig = px.bar(y = sorty, #stock_list_data["33SectorName"],
             x = sortx, #stock_list_data['total'] , 
             orientation = 'h',
             color =stock_list_data["33SectorName"], #stock_list_data["33SectorName"] ,
             color_continuous_scale="Viridis",
             #barmode="group",
             width=1600, height=3000) 
fig.update_xaxes(title="IssuedShares")
fig.update_yaxes(title = "Type of Sector")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution IssuedShares ',
        'xanchor': 'center',
        'yanchor': 'top'} ,
        template="plotly_white")
fig.show()


In [ ]:
stock_list_data_corr = stock_list_simple.corr()
fig = px.imshow(stock_list_data_corr, text_auto=True)
fig.show()

In [ ]:

#merged_stock_price.style.background_gradient(cmap='gist_rainbow_r')
stock_list[0:40].style.background_gradient(cmap='gist_rainbow_r')

In [ ]:
try:
    from finta import TA
    from backtesting import Backtest, Strategy
    from backtesting.lib import crossover
except:
    !pip install finta backtesting
    from finta import TA
    from backtesting import Backtest, Strategy
    from backtesting.lib import crossover

In [ ]:
function_dict = {' Simple Moving Average ' : 'SMA',
                 ' Simple Moving Median ' : 'SMM',
                 ' Smoothed Simple Moving Average ' : 'SSMA',
                 ' Exponential Moving Average ' : 'EMA',
                 ' Double Exponential Moving Average ' : 'DEMA',
                 ' Triple Exponential Moving Average ' : 'TEMA',
                 ' Triangular Moving Average ' : 'TRIMA',
                 ' Triple Exponential Moving Average Oscillator ' : 'TRIX',
                 ' Volume Adjusted Moving Average ' : 'VAMA',
                 ' Kaufman Efficiency Indicator ' : 'ER',
                 ' Kaufmans Adaptive Moving Average ' : 'KAMA',
                 ' Zero Lag Exponential Moving Average ' : 'ZLEMA',
                 ' Weighted Moving Average ' : 'WMA',
                 ' Hull Moving Average ' : 'HMA',
                 ' Elastic Volume Moving Average ' : 'EVWMA',
                 ' Volume Weighted Average Price ' : 'VWAP',
                 ' Smoothed Moving Average ' : 'SMMA',
                 ' Fractal Adaptive Moving Average ' : 'FRAMA',
                 ' Moving Average Convergence Divergence ' : 'MACD',
                 ' Percentage Price Oscillator ' : 'PPO',
                 ' Volume-Weighted MACD ' : 'VW_MACD',
                 ' Elastic-Volume weighted MACD ' : 'EV_MACD',
                 ' Market Momentum ' : 'MOM',
                 ' Rate-of-Change ' : 'ROC',
                 ' Relative Strength Index ' : 'RSI',
                 ' Inverse Fisher Transform RSI ' : 'IFT_RSI',
                 ' True Range ' : 'TR',
                 ' Average True Range ' : 'ATR',
                 ' Stop-and-Reverse ' : 'SAR',
                 ' Bollinger Bands ' : 'BBANDS',
                 ' Bollinger Bands Width ' : 'BBWIDTH',
                 ' Momentum Breakout Bands ' : 'MOBO',
                 ' Percent B ' : 'PERCENT_B',
                 ' Keltner Channels ' : 'KC',
                 ' Donchian Channel ' : 'DO',
                 ' Directional Movement Indicator ' : 'DMI',
                 ' Average Directional Index ' : 'ADX',
                 ' Pivot Points ' : 'PIVOT',
                 ' Fibonacci Pivot Points ' : 'PIVOT_FIB',
                 ' Stochastic Oscillator Percent K ' : 'STOCH',
                 ' Stochastic oscillator Percent D ' : 'STOCHD',
                 ' Stochastic RSI ' : 'STOCHRSI',
                 ' Williams Percent R ' : 'WILLIAMS',
                 ' Ultimate Oscillator ' : 'UO',
                 ' Awesome Oscillator ' : 'AO',
                 ' Mass Index ' : 'MI',
                 ' Vortex Indicator ' : 'VORTEX',
                 ' Know Sure Thing ' : 'KST',
                 ' True Strength Index ' : 'TSI',
                 ' Typical Price ' : 'TP',
                 ' Accumulation-Distribution Line ' : 'ADL',
                 ' Chaikin Oscillator ' : 'CHAIKIN',
                 ' Money Flow Index ' : 'MFI',
                 ' On Balance Volume ' : 'OBV',
                 ' Weighter OBV ' : 'WOBV',
                 ' Volume Zone Oscillator ' : 'VZO',
                 ' Price Zone Oscillator ' : 'PZO',
                 ' Elders Force Index ' : 'EFI',
                 ' Cummulative Force Index ' : 'CFI',
                 ' Bull power and Bear Power ' : 'EBBP',
                 ' Ease of Movement ' : 'EMV',
                 ' Commodity Channel Index ' : 'CCI',
                 ' Coppock Curve ' : 'COPP',
                 ' Buy and Sell Pressure ' : 'BASP',
                 ' Normalized BASP ' : 'BASPN',
                 ' Chande Momentum Oscillator ' : 'CMO',
                 ' Chandelier Exit ' : 'CHANDELIER',
                 ' Qstick ' : 'QSTICK',
                 #' Twiggs Money Index ' : 'TMF',
                 ' Wave Trend Oscillator ' : 'WTO',
                 ' Fisher Transform ' : 'FISH',
                 ' Ichimoku Cloud ' : 'ICHIMOKU',
                 ' Adaptive Price Zone ' : 'APZ',
                 #' Squeeze Momentum Indicator ' : 'SQZMI',
                 ' Volume Price Trend ' : 'VPT',
                 ' Finite Volume Element ' : 'FVE',
                 ' Volume Flow Indicator ' : 'VFI',
                 ' Moving Standard deviation ' : 'MSD',
                 ' Schaff Trend Cycle ' : 'STC'}


In [ ]:
stock_price = Train_Stock_File_Specs["Stock_Price_File"]
for code in stocks[0:10]:
    stock_name = merged_stock_price[merged_stock_price["SecuritiesCode"] == code]['Name'].unique()
    stock_price_data = merged_stock_price[merged_stock_price["SecuritiesCode"] == code]
    stock_price_data['Date'] = pd.to_datetime(stock_price_data.Date, utc=False)
    stock_price_data.index = pd.DatetimeIndex(stock_price_data['Date'])
    ohlc=stock_price_data #[-10000:]
    #print(ohlc.head())
    for key, value in function_dict .items():
        function_name = "TA." + value + "(ohlc).plot(title='" + key + " for " + str(code)  + "')"
        #print(function_name)
        result = eval(str(function_name))

In [ ]:
import plotly.graph_objects as go
stock_price = Train_Stock_File_Specs["Stock_Price_File"]
for code in stocks[100:110]:
    stock_name = merged_stock_price[merged_stock_price["SecuritiesCode"] == code]['Name'].unique()
    title_text = str('Japan Stock Exchange Stock Summary for ' + stock_name)
    yaxis_text = str(stock_name)
    stock_price_data = merged_stock_price[merged_stock_price["SecuritiesCode"] == code]
    hovertext=[]
    fig = go.Figure(data=go.Ohlc(x=stock_price_data['Date'],
                                 open=stock_price_data['Open'],
                                 high=stock_price_data['High'],
                                 low=stock_price_data['Low'],
                                 close=stock_price_data['Close'],
                                 text=hovertext,
                                 hoverinfo='text'))
    fig.update_layout(title=title_text,
                     yaxis_title=yaxis_text,
                     )
    fig.show()

In [ ]:
stock_price = Train_Stock_File_Specs["Stock_Price_File"]
for code in stocks[240:250]:
    stock_name = merged_stock_price[merged_stock_price["SecuritiesCode"] == code]['Name'].unique()
    stock_price_data = merged_stock_price[merged_stock_price["SecuritiesCode"] == code]
    stock_price_data['Date'] = pd.to_datetime(stock_price_data.Date, utc=False)
    stock_price_data.index = pd.DatetimeIndex(stock_price_data['Date'])
    ohlc=stock_price_data #[-10000:]
    #print(ohlc.head())
    for key, value in function_dict .items():
        title_text = str('Japan Stock Exchange Stock Summary for ' + stock_name)
        yaxis_text = str(stock_name)
        #ohlc=ohlc.reset_index()
        ohlc['SMA5'] = ohlc.Close.rolling(5).mean()
        ohlc['SMA20'] = ohlc.Close.rolling(20).mean()
        ohlc['SMA50'] = ohlc.Close.rolling(50).mean()
        ohlc['SMA200'] = ohlc.Close.rolling(200).mean()
        ohlc['SMA500'] = ohlc.Close.rolling(500).mean()
        ohlc['EMA5'] = ohlc.Close.ewm(span=5, adjust=False).mean()
        ohlc['EMA20'] = ohlc.Close.ewm(span=20, adjust=False).mean()
        ohlc['EMA50'] = ohlc.Close.ewm(span=50, adjust=False).mean()
        ohlc['EMA200'] = ohlc.Close.ewm(span=200, adjust=False).mean()
        ohlc['EMA500'] = ohlc.Close.ewm(span=500, adjust=False).mean()
        fig = go.Figure(data=[go.Ohlc(x=ohlc['Date'],
                                  open=ohlc['Open'],
                                  high=ohlc['High'],
                                  low=ohlc['Low'],
                                  close=ohlc['Close'], name = "OHLC"),
                              go.Scatter(x=ohlc.Date, y=ohlc.EMA5, line=dict(color='orange', width=1), name="EMA5"),
                              go.Scatter(x=ohlc.Date, y=ohlc.EMA20, line=dict(color='green', width=1), name="EMA20"),
                              go.Scatter(x=ohlc.Date, y=ohlc.EMA50, line=dict(color='blue', width=1), name="EMA50"),
                              go.Scatter(x=ohlc.Date, y=ohlc.EMA200, line=dict(color='violet', width=1), name="EMA200"),
                              go.Scatter(x=ohlc.Date, y=ohlc.EMA500, line=dict(color='purple', width=1), name="EMA500"),
                              go.Scatter(x=ohlc.Date, y=ohlc.SMA5, line=dict(color='navy', width=1), name="SMA5"),
                              go.Scatter(x=ohlc.Date, y=ohlc.SMA20, line=dict(color='lime', width=1), name="SMA20"),
                              go.Scatter(x=ohlc.Date, y=ohlc.SMA50, line=dict(color='fuchsia', width=1), name="SMA50"),
                              go.Scatter(x=ohlc.Date, y=ohlc.SMA200, line=dict(color='indigo', width=1), name="SMA200"),
                              go.Scatter(x=ohlc.Date, y=ohlc.SMA500, line=dict(color='black', width=1), name="SMA500")])
        fig.update_layout(title=title_text,yaxis_title=yaxis_text,)
        
        fig.show()

# Work In Progress